# A DNA-Based Archival Storage System

In this assignment, we will presents an architecture for a DNA-based archival storage system. It is structured as a key-value store, and leverages common biochemical techniques to provide random access. We also propose a new encoding scheme that offers controllable redundancy, trading off reliability for
density.

## Overview

A DNA storage system consists of a DNA synthesizer that encodes the data to be stored in DNA, a storage container with compartments that store pools of DNA that map to a volume, and a DNA sequencer that reads DNA sequences
and converts them back into digital data.


## ✍️ Contributors: Muhammad Dahamshi & Firas Ramadan

### Importing Packages & utilities
Importing python packages as well as declaring configuration parameters and auxiliary functions.

In [0]:
import numpy as np
import pandas as pd
import hashlib 
from termcolor import colored
import matplotlib.pyplot as plt
from urllib.request import urlopen  # the lib that handles the url stuff
from typing import List
import math 
import textwrap
import random
import copy
import progressbar
from time import sleep
## This line makes matplotlib plot the figures inside the notebook
%matplotlib inline
primer_len = 10
high_address_len = 5
low_address_len = 15
pay_load_len = 80
dna_library_size = 40

#### https://convertio.co/bin-jpg/ in order to convert the binary file to jpg files
	

# utilities

upside_down_letters = {'A':'ꓯ','C':'Ↄ','G':'⅁','T':'ꓕ'}
letters_complement = {'A':'T','C':'G','G':'C','T':'A'}

def substract_lists(A,B):
   return [item for item in A if item not in B]

def upside_down_complement(nucleotide):
  #return upside_down_letters[letters_complement[nucleotide]]
  return letters_complement[nucleotide]

def rotate(l, n):
    return l[n:] + l[:n]

def ternary(n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
    return ''.join(reversed(nums))

def str_to_bin(string: str) -> str:
    return ''.join(format(ord(n), 'b') for n in string)

def byte_to_bin(byte) -> str:
    return ''.join(format(ord(byte), 'b').zfill(8) ) if len(byte) > 0 else ''

def sub_str_from_indices(string: str, indeces) -> str:
    return ''.join(string[i] for i in indeces)

def get_as_ternary(string: str) -> str:
    return ternary(int(string))

def print_aux(string: str, color):
        print(colored('\033[1m' + string, color), end='')
def print_seq(seq: str, data_len=pay_load_len):   
    n1 = 0
    n2 = primer_len
    n3 = n2 + 1
    n4 = n3 + data_len
    n5 = n4 + low_address_len + high_address_len;
    n7 = primer_len + data_len + 1
    print_aux(seq[n1:n2], 'red')
    print_aux(seq[n2], 'blue')
    print_aux(seq[n3:n4], 'magenta')
    print_aux(seq[n4:n5], 'yellow')
    print_aux(seq[n5], 'blue')
    print_aux(seq[-n2:], 'red')
    print('')

def print_colored(sequences: list):  
    for i in range(len(sequences) - 1):
        seq = sequences[i][0]
        complement = sequences[i][1]
        print_seq(seq)
        print_seq(complement)
        print('')
    i = len(sequences) - 1
    seq = sequences[i][0]
    complement = sequences[i][1]
    data_len = (len(seq) - (2 * primer_len + 2 + low_address_len + high_address_len))
    print_seq(seq,data_len)
    print_seq(complement,data_len)

def split_into_fragments(string,frag_len):
    return textwrap.wrap(string, frag_len)

def get_complement_seq(sequences):
    return ''.join(list(map(upside_down_complement,sequences)))


## Huffman code, A method for the construction of minimum redundancy codes
Huffman code that maps each binary byte to either 5 or 6 ternary digits. For
example, the Huffman code maps the binary string 01100001 to the base-3 string 01112. The rotating nucleotide encoding maps this string to the DNA sequence CTCTG. The code maps more common ASCII characters to 5 digit strings, offering
minor compression benefits for textual data, though the effect on overall storage density is insignificant.

This part is resposible for the translation of the received binary code to huffman code using the translation table that is given below.


In [0]:
huffman_path = 'https://raw.githubusercontent.com/Muhammad-Dahamshi/DNA-Storage-System/master/dna/data/huff3.dict.csv'
## Loading the data, the using of converters is to keep leading zeros in a column when reading CSV,
## thus,we have to prevent slicing of '00100' to '100' for example
huffman_dict = pd.read_csv(huffman_path, converters={'code': lambda x: str(x)})

## Show the first 100 rows
huffman_dict.head(100)

,number,code
0,0,22201
1,1,00100
2,2,11220
3,3,00211
4,4,20222
...,...,...
95,95,11201
96,96,00102
97,97,01112
98,98,22010


This part includes functions that translate the binary code to ternary code and vice versa according to what we've called huffman dictionary ( translation table ) 


In [0]:
huffman_array = huffman_dict[['number', 'code']].values 	


#print(huffman_array2[0:10])

def huffman_code(x: int) -> str:
        return str(huffman_array[x][1]) if (x >= 0 & x <= 255) else ''

def get_huffman_index(string:str):
		for j in range(len(huffman_array)):
				if huffman_array[j][1] == string:
					return j 
		return -1

def huffman_to_binary(huffman):
		str_res = ''
		index = 0
		while len(huffman)>0:
			str_to_convert = huffman[0:5]
			num = get_huffman_index(str_to_convert)
			if(num == -1):
				str_to_convert = huffman[0:6]
				num = get_huffman_index(str_to_convert)
			huffman = huffman[len(str_to_convert):]
			str_res = str_res + format(num, 'b').zfill(8)
		return str_res


print('Huffman code of number 5\t = {}'.format(huffman_code(5)))
print('Huffman code of number 250\t = {}'.format(huffman_code(250)))
print('Huffman code of number 97\t = {}'.format(huffman_code(97)))
print('Huffman code of number 94\t = {}'.format(huffman_code(94)))


Huffman code of number 5	 = 00222
Huffman code of number 250	 = 22020
Huffman code of number 97	 = 01112
Huffman code of number 94	 = 222211


## 📚 Huffman To Nucleotides

✍️ A rotating encoding to nucleotides avoids homopolymers (repetitions
of the same nucleotide), which are error-prone

# A comment about NumPy indexing
![alt text](https://raw.githubusercontent.com/Muhammad-Dahamshi/DNA-Storage-System/master/pdf-images/rotating-encoding.png)


As explained in the article, at the end what interests us is having nucleotides that could build a synthetic DNA while maintaining the lowest error rate possible, to do so we used the following function which translates the huffman codes to nucleotides while using a translation table that its goal is to prevent the repitition of the same nucleotides, a thing that increases the possiblities for errors. 

In [0]:
map_letters = ['A','C','G','T']
def huffman_to_nucleotides(huff_num: str, previous='A') -> str:
        nucleotides = ''
        previous_letter = previous
        base =['C','G', 'T', 'A'] 
        converter = [rotate(base,i) for i in range(3)]
        for i in huff_num:
            nucleotide = converter[int(i)][map_letters.index(previous_letter)]
            nucleotides += nucleotide
            previous_letter = nucleotide 
        return nucleotides

def nucleotides_to_huffman (nucleotides):
		huffman = ''
		previous_letter = 'A'
		base = map_letters
		converter = {'A': rotate(base,0)[1:] , 'C':  rotate(base,1)[1:]
		            ,'G': rotate(base,2)[1:], 'T':  rotate(base,3)[1:]}
		for i in range(len(nucleotides)):
				cur = nucleotides[i]
				huffman += str(converter[previous_letter].index(cur))
				previous_letter = nucleotides[i]
		return huffman
		

# Data Keys to nucleotides Primers
The next part is responsible for applying the known hash function SHA-256 
on the file/data keys that we are interested in reading/writing from/to the right DNA pool. Then it translates the hash function's outcome to nucleotides using the function and the translations we declared before.

In [0]:
def map_key_to_primer(key_string):
    sha_signature = \
        hashlib.sha256(key_string.encode()).hexdigest()
    sha_to_bin = str_to_bin(sha_signature)
    bin_to_ternary = get_as_ternary(sha_to_bin)
    n1 = bin_to_ternary[:(primer_len)]
    n2 = bin_to_ternary[(-1*primer_len):]
    primer1 = huffman_to_nucleotides(n1)
    primer2 = huffman_to_nucleotides(n2)
    return [primer1,primer2]

print(map_key_to_primer('foo.txt'))
print(map_key_to_primer('goo.txt'))
print(map_key_to_primer('doo.txt'))
print(map_key_to_primer('Firas'))
print(map_key_to_primer('Muhammad'))

['GTGACTGATC', 'GCGTAGTCTC']
['GTACACTGAC', 'CTGTAGTCTA']
['TAGTGCTAGT', 'TCATGAGTGT']
['GAGTGTGACT', 'GCATGAGAGA']
['GTACACTACT', 'TAGCATACAG']


# Data keys to High Bits of Address
similarly to the previous section this part will take the data/file keys and translates them to nucleotides using different hash function and the same huffman to nucleotides translation that we used before. 
After this process is done those nucleotides which are the high bits of the fragements' addresses will be appended to the lower bits (will be explained in the next section) and together appended to the rest of the data and primers nucleotides.


In [0]:
def rand_indices(string: str):
    ## Create a random generator using a fixed seed
    rand_gen = np.random.RandomState(0)
    ## Generating a vector of indices
    indices = np.arange(0, len(string), 1)
    ## Shuffle the indices
    rand_gen.shuffle(indices)
    return indices


def map_key_to_high_address(key_string):
    key_to_bin = str_to_bin(key_string)
    bin_to_ternary = get_as_ternary(key_to_bin)
    indices=rand_indices(bin_to_ternary)[0:high_address_len]
    address = huffman_to_nucleotides(sub_str_from_indices(bin_to_ternary,indices))
    return address

def map_index_to_low_address(index):
    low_address_len_aux = low_address_len * 2
    to_ternary = ternary(index).rjust(low_address_len_aux, '2')
    address = huffman_to_nucleotides(to_ternary)[-low_address_len:]
    return address

def map_key_to_pool_address(dna_library, key_string):
    return hash(key_string) % (len(dna_library))


print(map_key_to_high_address('foo.txt'))
print(map_key_to_high_address('fpo.txt'))
print(map_key_to_high_address('foo.txt'))


def map_strand_to_RDP_table(key , index):
    high_address = map_key_to_high_address(key)
    val_hex = hashlib.sha256(high_address.encode()).hexdigest()
    
    # Get integer value of digest from the hexdigest
    val_int = int(val_hex, 16) 
    #print('%064x' % val_int)   
    map_val = val_int + int(index/4) ## base + offset
    return map_val



for i in range(20):
    print(map_strand_to_RDP_table('foo.txt' , i))



CTGTA
GATGT
CTGTA
94798299524441165247921464256416679112784556086823442995666859768943701425625
94798299524441165247921464256416679112784556086823442995666859768943701425625
94798299524441165247921464256416679112784556086823442995666859768943701425625
94798299524441165247921464256416679112784556086823442995666859768943701425625
94798299524441165247921464256416679112784556086823442995666859768943701425626
94798299524441165247921464256416679112784556086823442995666859768943701425626
94798299524441165247921464256416679112784556086823442995666859768943701425626
94798299524441165247921464256416679112784556086823442995666859768943701425626
94798299524441165247921464256416679112784556086823442995666859768943701425627
94798299524441165247921464256416679112784556086823442995666859768943701425627
94798299524441165247921464256416679112784556086823442995666859768943701425627
94798299524441165247921464256416679112784556086823442995666859768943701425627
9479829952444116524792146425641667911278455608

#Digital data to "nucleotides data"
Using the translation tables and functions from the previous sections, this function converts the digital data of the input file to nucleotides.

In [0]:
def digital_to_nucleotides(binary: str):
    nucleotides_res = ""
    for i in np.arange(0, len(binary), 8):
        tmp_result = int(binary[i:i + 8], 2)
        nucleotides_res += (huffman_code(tmp_result))
    return huffman_to_nucleotides(nucleotides_res)

def nucleotides_to_digital(sequence: str):
    return huffman_to_binary(nucleotides_to_huffman(sequence))   


binary_test = '010100000110111101101100011110010110000100111011'
print(digital_to_nucleotides(binary_test))
#GCGAGTGAGTATCGATGCTCTAGAGCATGTGA

GCGAGTGAGTATCGATGCTCTAGAGCATGTGA


#Encode into Fragments
In this section the file is first read, and then encoded from binary code to nucleotides data, which will be splitted into fragments according to the data's length and the number of the nucleotides that represent the digital data.

In [0]:
dir_path = "https://raw.githubusercontent.com/Muhammad-Dahamshi/DNA-Storage-System/master/examples/"
cat_key = 'cat.jpg'
smiley_key = 'smiley.jpg'
sydney_key = 'sydney.jpg'
txt_file_key = 'ex.txt'
shakespeare_key = 'wssnt10.txt'
corbett_key =  'corbett.pdf'
small_key =  'result2.png'

cat_path = dir_path + cat_key
smiley_path = dir_path + smiley_key
sydney_path = dir_path + sydney_key
txt_file_path = dir_path + txt_file_key
shakespeare_path = dir_path + shakespeare_key
#reading the binary file
def get_file_stream(input_file):
    open_file_string = urlopen(input_file).read()
    return ''.join(format(n, 'b').zfill(8) for n in open_file_string)
# converting the digital data and split it into fragments
def encode_payload(input_file, frag_len=pay_load_len):
    lst = []
    current_str = ''
    digital_val = ''
    open_file = urlopen(input_file)
    stream =  get_file_stream(input_file)
    stream_to_nucleotides = digital_to_nucleotides(stream)
    return split_into_fragments(stream_to_nucleotides,frag_len)


#Assembling Sequences
These functions are responsible to gather all the translated nucleotides and appended in the right order in order to put it in the DNA pool.
The primers are placed at the start and the end of each strand (we assumed that each primer's length is fixed to 10 nucleotides)
the data nucleotides or what we've called the payload is appended to the address ( both high and low bits ) and then all assembled in the
right order forming a strand orginated in the following way:

***|Start_primer(10nucs)|sense_nuc(1nuc)|payload(80nucs)|high_address(5nucs)|low_address(15nucs)|sense_nuc|End_Primer(10nucs)|***
![alt text](https://raw.githubusercontent.com/Muhammad-Dahamshi/DNA-Storage-System/master/examples/result2.png)


In [0]:
def append_payload_to_address(sequences_list: List[str], high_address):
    lst = []
    for i in range(len(sequences_list)):
        low_address = map_index_to_low_address(i)
        lst.append(sequences_list[i] + high_address + low_address)
    return lst

def assemble(sequences_list: List[str], primer1, primer2, high_address):
    address_append_map = append_payload_to_address(sequences_list, high_address)
    def append_primers_and_sense(seq, sense_nucleotides='A'):
        return primer1 + sense_nucleotides + seq + sense_nucleotides + primer2
    assembled = list(map(append_primers_and_sense, address_append_map))
    complement_attach = [[strand,get_complement_seq(strand)] for strand in assembled]
    return complement_attach

def encode(key,rdp_library={},with_replication = False):
    full_path = dir_path + key
    primers = map_key_to_primer(key)
    primer1 = primers[0]
    primer2 = primers[1]
    high_address = map_key_to_high_address(key)
    encoded_payload = encode_payload(full_path)
    
    if with_replication:
      x = len(encoded_payload)
      to_save = (int(x/4)-1)*4 if (int(x/4)*4 == x) else (int(x/4))*4
      for i in range(0,to_save,4):
        table_i = RDP_table_from_strands(encoded_payload[i],encoded_payload[i+1],
                              encoded_payload[i+2],encoded_payload[i+3])
        index = map_strand_to_RDP_table(key,i)
        rdp_library[str(index)] = table_i
        #print_RDP_table(table_i)
      #print(len(rdp_library))
    assemble_sequences = assemble(encoded_payload, primer1, primer2, high_address)
    return assemble_sequences

# keys_list = [cat_key,smiley_key,sydney_key,txt_file_key,shakespeare_key]

#print_colored(encode(smiley_key))
#print(len(encode(cat_key)))
#print(len(encode(smiley_key)))
#print(len(encode(sydney_key)))
#print(len(encode(txt_file_key)))
#print(len(encode(shakespeare_key)))
#print(len(encode(corbett_key)))
print(len(encode(small_key))) #483

x = len(encode(small_key))
to_save = (int(x/4)-1)*4 if (int(x/4)*4 == x) else (int(x/4))*4
print(to_save)
#print_colored(encode(small_key))
#print_colored(encode(txt_file_key))

483
480


Smiley.png result

![alt text](https://raw.githubusercontent.com/Muhammad-Dahamshi/DNA-Storage-System/master/examples/result1.png)


#Functions used in different sections:
###Insert:
 after inserting the "half DNA" to the pool we append its complementary nucleotides for each strand so it will be a full DNA strand.
###get_payload:
This function extracts the payload nucleotides (data) from the full DNA strand in order to decode it and convert it back to binary code.
###Search:
uses the primers that it receives in order to find the needed strands from the right pool.
###Create_DNA_Library:
creates a group of pools that will be used to save data in. The right pool will be chosen using hash functions on the high bits of each file's address.

In [0]:
def insert(pool, strand_complement_pair):
    pool.append(strand_complement_pair)

def get_payload(strand: str):
    nuc_to_cut_start = primer_len+1
    nuc_to_cut_end = primer_len+low_address_len+high_address_len+1
    payload = strand[nuc_to_cut_start:len(strand)-nuc_to_cut_end]
    return payload

def search(pool, primer1, primer2, length=primer_len):
    lst = []
    for strand_complement_pair in pool:
        strand = strand_complement_pair[0]
        if (strand[0:length] == primer1) \
                & (strand[-length:] == primer2):
            lst.append(strand)
    return lst

def create_dna_library(size=dna_library_size):
    lst = []
    for i in range(size):
        lst.append([])
    return lst

def RDP_library(size=dna_library_size):
    dict = {}
    for i in range(size):
        lst.append([])
    return lst

    


# Row-Diagonal Parity

### Row-Diagonal Parity auxiliary & utilities

In [0]:

nucleotides_dict = [ 'A' , 'C' , 'G' , 'T' ]

nuc_to_num = lambda x : nucleotides_dict.index(x)
num_to_nuc = lambda index : nucleotides_dict[index]
flatten = lambda l: [item for sublist in l for item in sublist]

diag_full_length = [5 , 5, 5, 5, 4]

def RDP_split(strand):
      return split_into_fragments(strand,int(len(strand)/4))

def create_RDP_table(p=5):
    lst = []
    for i in range(p-1):
      row = []
      for j in range(p+1):
        row.append([])
      lst.append(row)  
    return lst

def print_RDP_table(table , print_title = True):
    data_num = len(table[0]) - 2 
    block_len = max(25,len(table[0][0][0])) if len(table[0][0])> 0 else 25
    if print_title:
      titles = ["data disk "+str(i) for i in range(data_num)]
      titles.extend(["Row Parity" , "Diagonal Parity"])
      for title in titles:
        print('{:^{len}}'.format(title,len=block_len),end = '')   
      print('')  
    for row in table:
        for elem in row:
          print('{:^{len}}'.format(str(elem),len=block_len),end = '')
        print('')  

def print_RDP_library(rdp_library):
    for k, v in rdp_library.items():
      print_aux('\n{:^{len}}\n'.format('{}'.format(k),len=120),'blue')
      print_RDP_table(v)  

def get_RDP_library_disks_state(rdp_library):
    all_disks = 0
    invalid_disks = 0
    for k, v in rdp_library.items():
      all_disks += 4
      invalid_disks += len(get_table_missed_disks(v))
    return invalid_disks,all_disks   

def print_rdp_library_state(rdp_library):
    invalid_disks,all_disks = get_RDP_library_disks_state(rdp_library)
    print('{}{}'.format('All disks\t',all_disks))
    valid_percentage = int((all_disks-invalid_disks)/all_disks * 100)
    print('{}{} ({}%)'.format('Valid disks :\t',all_disks-invalid_disks,valid_percentage))
    if(invalid_disks == 0):
      print_aux('All data is valid\n\n','blue')
    if(invalid_disks != 0):
      answer = input("print failed desks info? enter yes or no\n")
      if answer == "yes":
          for k, v in rdp_library.items():
            missed = get_table_missed_disks(v)
            if missed != [] :
              print_aux('disk uuid: {}\t{}\n'.format(k,'failed Desks {}'.format(missed)),'blue')
      else:
          return

def get_table_missed_disks(table):
    lst = []
    data_num = len(table[0]) - 2
    
    for j in range(data_num+2):
      for i in range(data_num):
        if table[i][j] == ['ERROR']:
          lst.append(j)
          break
    return lst

def is_valid(table):
    data_num = len(table[0]) - 2
    for i in range(data_num):
      for j in range(data_num+2):
        if table[i][j] == ['ERROR']:
          return False
    return True      


Row-Diagonal Parity data reconstruction

In [0]:

def diag_row_distribution(data_num):
    indexes = []
    base = list(range(data_num+1))
    for i in range(data_num):
      diag = rotate(base,i)
      diag.append(i)
      indexes.append(diag)
    return indexes  

def diag_indexes(data_num,diag, with_diag_parity = True):
    lst = []
    indexes = diag_row_distribution(data_num)
    last = data_num+2 if with_diag_parity else data_num+1
    for col in range(last):
      for row in range(data_num):
        if indexes[row][col]==diag:
          lst.append([col,row])
    return lst  


def table_get_diag(table , diag  , with_diag_parity = True):
      diag_lst = []
      data_num = len(table[0]) - 2
      indexes = diag_indexes(data_num,diag,with_diag_parity)
      for pair in indexes:
        col = pair [0]
        row = pair [1]
        if table[row][col] != ['ERROR']: 
          diag_lst.append(table[row][col])
      return diag_lst 


def nucleotides_XOR(*nucleotides):
    def XOR_2(nuc1,nuc2):
        assert len(nuc1) == len(nuc2)
        nucs_len = len(nuc1)
        #print(nuc1,nuc2, sep =' ')
        return ''.join(num_to_nuc(nuc_to_num(nuc1[i])^nuc_to_num(nuc2[i])) for i in range(nucs_len))
    nuc_len = len(nucleotides[0])
    xor_res = "A" * nuc_len
    for nuc in nucleotides: 
        xor_res = XOR_2(xor_res,nuc)
    return xor_res

def table_calc_diag_parity(table , diag):
    diag_lst = table_get_diag(table , diag)
    return nucleotides_XOR(*(flatten(diag_lst)))

def table_calc_row_parity(table , row):
    data_num = len(table[0]) - 2
    return nucleotides_XOR(*(flatten(table[row][0:data_num])))

def row_parity_fill(table):
    data_num = len(table[0]) - 2
    for row in range(data_num):
        table[:][row][data_num].insert(0,table_calc_row_parity(table,row))     

def diag_parity_fill(table):
    data_num = len(table[0]) - 2
    for diag in range(data_num): # the diagonal for which we do not store parity is the last (diag = 4)
        table[:][diag][data_num + 1].insert(0,table_calc_diag_parity(table,diag))

def down_disk(table,disk_index):
    data_num = len(table[0]) - 2
    for i in range(data_num):
      table[:][i][disk_index] = ['ERROR']
              

def RDP_table_from_strands(*strands): #each strand is in a column (4 strands)
    lst = []
    for strand in strands:
      lst.append(RDP_split(strand))
    #print(lst)
    table = create_RDP_table()
    data_num = len(table[0]) - 2
    for i in range(data_num):
      for j in range(data_num):
        table[i][j].insert(0,lst[j][i])
    row_parity_fill(table)
    diag_parity_fill(table)
    return table  

def RDP_table_from_strands(*strands): #each strand is in a column (4 strands)
    lst = []
    for strand in strands:
      lst.append(RDP_split(strand))
    #print(lst)
    table = create_RDP_table()
    data_num = len(table[0]) - 2
    for i in range(data_num):
      for j in range(data_num):
        table[i][j].insert(0,lst[j][i])
    row_parity_fill(table)
    diag_parity_fill(table)
    return table


def RDP_recover_block_based_in_diag(table,diag,block_indexes_pair):
    # A ^ B ^ C ^ D = E
    # A missed -> new_A = E ^ B ^ C ^ D = (A ^ B ^ C ^ D)^ B ^ C ^ D 
    # E missed -> new_E = A ^ B ^ C ^ D = (A ^ B ^ C ^ D) 
    col = block_indexes_pair[1]
    row = block_indexes_pair[0]
    data_num = len(table[0]) - 2
    
    nucleotides = flatten(table_get_diag(table,diag))
    diag_xor_calc = nucleotides_XOR(*(nucleotides))
    new_block_data = diag_xor_calc
    
    if(len(table[row][col]) > 0): # remove invalid data (refresh)
        table[:][row][col].pop()
    table[:][row][col].insert(0,new_block_data)

def RDP_recover_block_based_in_row(table,block_indexes_pair):
    # A ^ B ^ C ^ D = E
    # A missed -> new_A = E ^ B ^ C ^ D = (A ^ B ^ C ^ D)^ B ^ C ^ D 
    # E missed -> new_E = A ^ B ^ C ^ D = (A ^ B ^ C ^ D) 
    col = block_indexes_pair[1]
    row = block_indexes_pair[0]
    data_num = len(table[0]) - 2

    valid_nucleotides = [] 
    for j in range(data_num+1):
        if table[row][j] != ['ERROR']: 
          valid_nucleotides.append(table[row][j])
          
    row_xor_calc = nucleotides_XOR(*(flatten(valid_nucleotides)))
    new_block_data = row_xor_calc
    if(len(table[row][col]) > 0): # remove invalid data (refresh)
        table[:][row][col].pop()
    table[:][row][col].insert(0,new_block_data)

def RDP_recover_completion(table,row):
    # A ^ B ^ C ^ D = E
    # A missed -> new_A = E ^ B ^ C ^ D = (A ^ B ^ C ^ D)^ B ^ C ^ D 
    # E missed -> new_E = A ^ B ^ C ^ D = (A ^ B ^ C ^ D) 

    data_num = len(table[0]) - 2
    valid_nucleotides = [] 
    for j in range(data_num+1):
        if table[row][j] != ['ERROR']:
          valid_nucleotides.append(table[row][j])
    if(len(valid_nucleotides) < data_num):
      raise         
    row_xor_calc = nucleotides_XOR(*(flatten(valid_nucleotides)))
    new_block_data = row_xor_calc
    for j in range(data_num+1):
        if table[row][j] == ['ERROR']:
            #print("old [{},{}]".format(row,j),table[row][j])
            table[:][row][j].pop() # remove invalid data (refresh)
            table[:][row][j].insert(0,new_block_data) 
            #print("new [{},{}]".format(row,j),table[row][j])

def RDP_recover_disk(table,disk_index):
    # A ^ B ^ C ^ D = E
    # A missed -> new_A = E ^ B ^ C ^ D = (A ^ B ^ C ^ D)^ B ^ C ^ D 
    # E missed -> new_E = A ^ B ^ C ^ D = (A ^ B ^ C ^ D) 
    # diag recover
    #disk_index =
    data_num = len(table[0]) - 2 
    indexes = diag_row_distribution(data_num)
    
    for diag in range(data_num): # list subtraction operation
      valid_data = table_get_diag(table,diag)
      full_diag = diag_full_length[diag]
      if(len(valid_data) == full_diag-1): # we can recover this diag
        for row in range(data_num):
          if((indexes[row][disk_index] == diag) & (table[row][disk_index] == ['ERROR'])):
            RDP_recover_block_based_in_diag(table,diag,[row,disk_index])
            RDP_recover_completion(table,row)
    return table



# Row-Diagonal Parity recover algorithm

In [0]:
def RDP_recover(table): #each strand is in a column (4 strands)
    lst = []
    data_num = len(table[0]) - 2
    missed_disks = get_table_missed_disks(table)
    if (len(missed_disks) >= 3):
      print('-'*150)  
      print('{:^{len}}'.format('CURRENT TABLE IS NOT RECOVERABLE',len=120))   
      print('-'*150)
      raise
    if (len(missed_disks) <= 0):
      return table
    recovred_table = table
    while(is_valid(recovred_table) != True):
      if(len(missed_disks) == 1):
        for row in range(data_num):
          disk = missed_disks[0]
          RDP_recover_block_based_in_row(recovred_table,[row,disk])
        return recovred_table 
      for disk in missed_disks:
        recovred_table = RDP_recover_disk(recovred_table,disk)
        missed_disks = get_table_missed_disks(recovred_table)
        if (len(missed_disks) <= 0):
          return recovred_table       
    return None # we shouldn't get here
 

# Row-Diagonal Parity TESTING

In [0]:
def utility_rand_fill():
    def rand_strand(length: int):
        ## Create a random generator using a fixed seed
        rand_gen = np.random.RandomState(0)
        ## Generating a list of random strand
        rand_strand = ''.join(random.choices(nucleotides_dict, k = length))
        #print(rand_strand)
        return rand_strand
    table = create_RDP_table()
    data_num = len(table[0]) - 2
    for i in range(data_num):
      for j in range(data_num):
          table[i][j].insert(0,rand_strand(20))
    return table

# TAGTATATCGACTAGTACAG
# AAAAAAAAAAAAAAAAAAAA
# CGCGCGCGCGCGCGCGCGCG
# CGCGCGCGCGCGCGCGCGCG
# TTTTTTTTTTTTTTTTTTTT
# RDP_table_from_strands('TAGTATATCGACTAGTACAG', 'CGTAGCATCTCGCAGCGAGA', 'TACGCTGCTACGCAGCATGC', 'TGTGAGTATCGATGACGAGT')

def unit_text_nuc_xor():
  print(nucleotides_XOR('A','C'))
  print(nucleotides_XOR('AAA','ACG'))
  print(nucleotides_XOR('C','G'))

def unit_test_simple():
    table = RDP_table_from_strands('AAAA', 'CGCG',
                               'CGCG', 'TTTT')
    print_RDP_table(table)
    print("diag 0: ",table_get_diag(table,0))   
    print("diag 1: ",table_get_diag(table,1))   
    print("diag 2: ",table_get_diag(table,2))   
    print("diag 3: ",table_get_diag(table,3))   
    print("diag 4: ",table_get_diag(table,4))
    data_num = len(table[0]) - 2
    print("\ndiag_row_indexes:\n",'\n'.join(str(a) for a in diag_row_distribution(data_num)),sep='')
  
def unit_test_layout_and_data():
    table = utility_rand_fill()
    print_RDP_table(table)

def unit_test_row_parity():
    table = utility_rand_fill()
    data_num = len(table[0]) - 2
    row_parity_fill(table)
    print_RDP_table(table)
    for row in range(data_num):
        row_parity = table[row][data_num][0]
        nucleotides = flatten(table[row][0:data_num])
        row_xor_calc = nucleotides_XOR(*(nucleotides))
        print_aux('SUCCESS\t','green') if row_parity == row_xor_calc \
        else print_aux('FAILED\t','red')
        print (" {} ==== {}{}\t".format(row_parity,nucleotides[0],
        ''.join(' ^ '+nuc for nuc in nucleotides[1:])))
        assert row_parity == row_xor_calc

def unit_test_diag_parity():
    table = utility_rand_fill()
    data_num = len(table[0]) - 2
    row_parity_fill(table)
    diag_parity_fill(table)
    print_RDP_table(table)   
    for diag in range(data_num): 
        diag_parity = table[diag][data_num+1][0]
        nucleotides = flatten(table_get_diag(table,diag,with_diag_parity=False))
        diag_xor_calc = nucleotides_XOR(*(nucleotides))
        print_aux('SUCCESS\t','green') if diag_parity == diag_xor_calc \
        else print_aux('FAILED\t','red')
        print (" {} ==== {}{}\t".format(diag_parity,nucleotides[0],
        ''.join(' ^ '+nuc for nuc in nucleotides[1:])))
        assert diag_parity == diag_xor_calc

def unit_test_RDP_table():
    table = RDP_table_from_strands('TAGTATATCGACTAGTACAG', 'CGTAGCATCTCGCAGCGAGA', 
                           'TACGCTGCTACGCAGCATGC', 'TGTGAGTATCGATGACGAGT')
    data_num = len(table[0]) - 2
    print_RDP_table(table)

    for i in range(data_num):
        row_parity = table[i][data_num][0]
        nucleotides = flatten(table[i][0:data_num])
        row_xor_calc = nucleotides_XOR(*(nucleotides))
        
        diag_parity = table[i][data_num+1][0]
        nucleotides = flatten(table_get_diag(table,i,with_diag_parity=False))
        diag_xor_calc = nucleotides_XOR(*(nucleotides))
        
        print_aux('SUCCESS\t','green') if row_parity == row_xor_calc \
        else print_aux('FAILED\t','red')
        print (" {} ==== {}{}\t".format(row_parity,nucleotides[0],
        ''.join(' ^ '+nuc for nuc in nucleotides[1:])))
        assert row_parity == row_xor_calc

        print_aux('SUCCESS\t','green') if diag_parity == diag_xor_calc \
        else print_aux('FAILED\t','red')
        print (" {} ==== {}{}\t".format(diag_parity,nucleotides[0],
        ''.join(' ^ '+nuc for nuc in nucleotides[1:])))
        assert diag_parity == diag_xor_calc


def unit_test_single_block_recover():
      print('-'*150)  
      table = utility_rand_fill()
      row_parity_fill(table)
      diag_parity_fill(table)
      print_RDP_table(table)
      valid = table[1][2]

      print_aux('\n{:^{len}}\n'.format('Diagonal parity reocover',len=120),'blue')

      table[1][2] = ['ERROR']
      print_RDP_table(table)
      RDP_recover_block_based_in_diag(table,diag = 3,block_indexes_pair=[1,2])
      print_RDP_table(table)
      print_aux('\n{:^{len}}\n'.format('SUCCESS',len=120),'green') if table[1][2] == valid \
        else print_aux('\n{:^{len}\n}'.format('FAILED',len=120),'red')

      print_aux('\n{:^{len}}\n'.format('Row parity reocover',len=120),'blue')
      table[1][2] = ['ERROR']
      print_RDP_table(table)
      RDP_recover_block_based_in_row(table,block_indexes_pair=[1,2])
      print_RDP_table(table) 
      print_aux('\n{:^{len}}\n'.format('SUCCESS',len=120),'green') if table[1][2] == valid \
        else print_aux('\n{:^{len}\n}'.format('FAILED',len=120),'red')
       
      print('-'*150)

def unit_test_single_desk_recover(disk_index = 1):
      print('-'*150)  
      table = utility_rand_fill()
      row_parity_fill(table)
      diag_parity_fill(table)
      down_disk(table,disk_index)
      print_aux('\n{:^{len}}\n'.format('Single Desk reocover',len=120),'blue')
      print_RDP_table(table)
      recovred = RDP_recover_disk(table,disk_index)
      print_RDP_table(recovred)
      print('-'*150)

def unit_test_two_desk_recover():
    def disks_list(): 
          sublist = [] 
          for i in range(4):  
              for j in range(i + 1, 4 , 1):
                  sub = [i,j] 
                  sublist.append(sub)  
          return sublist
    def test_aux(missed):       
          print('-'*150)
          print_aux('\n{:^{len}}\n'.format('Two Desks {} reocover'.format(missed),len=120),'blue')  
          table = RDP_table_from_strands('TCATGCACGTACATCTCGATCATAGCATGTACATCATGCACACTACTACAGTCGCGTATGCAGTGTCAGACAGACTGCGT'
,'ATACTGATAGCTGTGCATGTCTCGAGAGCAGCATCACACGCAGCGTCGTGTACAGCACGCTGCTGTGTATGCGCGTATCA'
,'TCTGCGCATGCGTCAGTGAGATCATGCGTGACGCTACGACTGATCTAGCTCGATCTAGCTCGATCTAGCTCGATCTAGCT'
,'CGATCTAGCTCGATCTAGCTCGATCTAGCTCGATCTAGCTCGATGCATAGAGCGCATGCACATGCAGATCGTACGTGTAC')    

          row_parity_fill(table)
          diag_parity_fill(table)
          print_RDP_table(table)
          for i in missed:
            down_disk(table,i)
          print_RDP_table(table)
          recovred = RDP_recover(table)
          print_RDP_table(recovred)
          print('-'*150)
     
    for m in disks_list():
       test_aux(m) 

def run_test(test):
    print('-'*150)  
    print('{:^{len}}'.format(test.__name__,len=120))   
    print('-'*150)
    test()

#run_test(unit_test_simple)
#run_test(unit_test_layout_and_data)
#run_test(unit_test_row_parity)
#run_test(unit_test_diag_parity)
#run_test(unit_test_RDP_table)
#run_test(unit_test_single_block_recover)
#run_test(unit_test_single_desk_recover)
#run_test(unit_test_two_desk_recover)


# PUT & GET
In a nutshell, those are the processes of read and write.
###Put:
This function receives all the DNA pools and the a key, using the key it will put the right strands (including the complementary nucleotides, meaning a full DNA strand) in the right pool.
###Get:
This function extracts the needed file from the pool and converts the nucleotides back to binary code using the methods that are mentioned in previous sections.

In [0]:
def put(key,dna_library, rdp_library = {} , with_replication = False):
    value = encode(key,rdp_library,with_replication)
    #print(len(rdp_library))
    pool_address = map_key_to_pool_address(dna_library, key)
    for strand_complement_pair in value:
        insert(dna_library[pool_address],strand_complement_pair)


def get(dna_library, key):
    primers = map_key_to_primer(key)
    primer1 = primers[0]
    primer2 = primers[1]
    pool_address = map_key_to_pool_address(dna_library, key)
    res = search(dna_library[pool_address], primer1, primer2)
    payloads = list(map(get_payload,search(dna_library[pool_address], primer1, primer2)))
    return nucleotides_to_digital(''.join(payload for payload in payloads))


#Main - Testing
Here we check our outcome on 5 different files (text files and images) 

In [0]:
# function to generate all the sub lists 
def sub_lists(list1): 
    # store all the sublists  
    sublist = [[]] 
    # first loop  
    for i in range(len(list1) + 1):          
        # second loop  
        for j in range(i + 1, len(list1) + 1):              
            # slice the subarray  
            sub = list1[i:j] 
            sublist.append(sub)  
    return sublist

def check(keys):
    if(len(keys)<=0):
      return
    # Declare a dictionary   
    dna_library = create_dna_library()  
    print("Testing #" + str(len(keys)) + ' keys\t:' + str(keys))
    for key in keys:
        put(key , dna_library)
    for key in keys:
        res1 = get_file_stream(dir_path+key)
        res2 = get(dna_library, key)
        print('\'' + key + '\'' + '\t:', end='')
        print_aux('SUCCESS\n','green') if res1 == res2 else print_aux('FAILED\n','red')
    print('---------------------------------------------------------------------')   


def check_recovery(key):
    rdp_library = {}
    dna_library = create_dna_library()
    put(key , dna_library, rdp_library , with_replication = True)
    valid_rdp = copy.deepcopy(rdp_library)
    _,all_disks = get_RDP_library_disks_state(rdp_library)
    # print_rdp_library_state(rdp_library)
    # print(len(rdp_library))
    for k, v in rdp_library.items():
      num1 = random.randint(0, 3)
      num2 = random.randint(0, 3)
      down_disk(v,num1)
      down_disk(v,num2)
    print_rdp_library_state(rdp_library)
    answer = input("\n\nprint all desks state? enter yes or no\n")
    if answer == "yes":
        print_RDP_library(rdp_library)
    print_aux('{}\n{:^{len}}\n{}\n'.format('*'*150,'data reocovering Start','*'*150,len=120), 'cyan')
    bar = progressbar.ProgressBar(maxval=all_disks, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    i = 1
    for k, v in rdp_library.items():
        i = i+1
        RDP_recover(v)
        bar.update(i)
    bar.finish()
    print_rdp_library_state(rdp_library)
    invalid_disks,all_disks = get_RDP_library_disks_state(rdp_library)
    print_aux('\n{:^{len}}\n'.format('check_recovery test result: SUCCESS\t',len=120),'green') if rdp_library == valid_rdp \
        else print_aux('\n{:^{len}}\n'.format('check_recovery test result: FAILED\t',len=120),'red')
    answer = input("\n\nprint recovered desks state? enter yes or no\n")
    if answer == "yes":
        print_RDP_library(rdp_library)


def main():
    keys_list = [cat_key,smiley_key,sydney_key,txt_file_key,shakespeare_key,small_key]
    keys_sub_lists = sub_lists(keys_list)
    keys_sub_lists.sort(key=len)
    #for sub_list in keys_sub_lists:
    #  check(sub_list)
    # check(keys_list)
    check_recovery(small_key)
if __name__ == '__main__':
    main()



All disks	480
Valid disks :	270 (56%)
print failed desks info? enter yes or no
no


print all desks state? enter yes or no
no


[========================================================================] 100%


******************************************************************************************************************************************************
                                                 data reocovering Start                                                 
******************************************************************************************************************************************************
All disks	480
Valid disks :	480 (100%)
All data is valid


                                          check_recovery test result: SUCCESS	                                          


print recovered desks state? enter yes or no
no
